# imports

In [ ]:
import tqdm
import pandas as pd
import numpy as np
from skimage.transform import resize
import os
import cv2
import matplotlib.pyplot as plt
import skimage
from collections import Counter
from skimage.morphology import thin
from PIL import Image
import tqdm

# define metrics

In [ ]:
def find_branchpoints(skeleton):
    #skeleton = skeleton.astype(int)
    return find_endpoints(skeleton) - 2

def find_endpoints(img):
    # Find row and column locations that are non-zero
    (rows,cols) = np.nonzero(img)

    # Initialize empty list of co-ordinates
    skel_coords = []

    # For each non-zero pixel...
    for (r,c) in zip(rows,cols):

        # Extract an 8-connected neighbourhood
        (col_neigh,row_neigh) = np.meshgrid(np.array([c-1,c,c+1]), np.array([r-1,r,r+1]))

        # Cast to int to index into image
        col_neigh = col_neigh.astype('int')
        row_neigh = row_neigh.astype('int')

        # Convert into a single 1D array and check for non-zero locations
        pix_neighbourhood = img[row_neigh,col_neigh].ravel() != 0

        # If the number of non-zero locations equals 2, add this to 
        # our list of co-ordinates
        if np.sum(pix_neighbourhood) == 2:
            skel_coords.append((r,c))

    return len(skel_coords)

def detect_fused(img):

    n_fused = 0
    n_single = 0

    img_thinned = thin(img) # or skeletonize, small difference
    img_thinned[0,:] = 0
    img_thin_labeled = skimage.measure.label(img_thinned.astype(np.uint8), connectivity=2)
    img_labeled = skimage.measure.label(img.astype(np.uint8), connectivity=2)
    stats_bbox = skimage.measure.regionprops(img_thin_labeled.astype(np.uint8))
    # results to fill
    fused_image = np.zeros_like(img)
    singles_image = np.zeros_like(img)
    finish = np.zeros_like(img)

    for i in range(0, len(stats_bbox)):

        bbox = stats_bbox[i].bbox
        # take thinned branch region
        bbox_region = img_thin_labeled[bbox[0]:bbox[2], bbox[1]:bbox[3]]

        # take its largest connected component in case multiple accidentally are in that bounding box
        value_counts = Counter(bbox_region.flatten()).most_common()
        most_frequent_value = value_counts[1][0] if len(value_counts) > 1 else value_counts[0][0]
        bbox_region = (bbox_region == most_frequent_value) * 1

        # if into that bounding box #branchpoints > 1 AND #endpoints >= 4, it is a FUSED filopodia
        bbox_region_padded = np.pad(bbox_region, pad_width=4, mode='constant', constant_values=0)
        n_endpoints = find_endpoints(bbox_region_padded)
        n_branchpoints = find_branchpoints(bbox_region_padded)
        is_fused = n_branchpoints > 1 and n_endpoints >= 4

        # mark FUSED and SINGLE regions with 2 different values
        if is_fused:
            fused_image += (img_labeled == (i + 1))
            n_fused += 1
        else:
            singles_image += (img_labeled == (i + 1))
            n_single += 1

        finish = singles_image + fused_image * 2

    return finish, n_single, n_fused


In [ ]:
def iou(prediction, true_mask):
    intersection = np.logical_and(prediction, true_mask).sum()
    union = np.logical_or(prediction, true_mask).sum()
    iou_score = intersection / union
    return iou_score

def dice(prediction, true_mask):
    intersection = np.logical_and(prediction, true_mask).sum()
    dice_score = (2. * intersection) / (prediction.sum() + true_mask.sum())
    return dice_score

def precision(prediction, true_mask):
    true_positives = np.logical_and(prediction, true_mask).sum()
    false_positives = np.logical_and(prediction, np.logical_not(true_mask)).sum()
    precision_score = true_positives / (true_positives + false_positives)
    return precision_score


def recall(prediction, true_mask):
    true_positives = np.logical_and(prediction, true_mask).sum()
    false_negatives = np.logical_and(np.logical_not(prediction), true_mask).sum()
    if int(true_positives + false_negatives) == 0:
        return 0
    recall_score = true_positives / (true_positives + false_negatives)
    return recall_score

def f1_score(prediction, true_mask):
    p = precision(prediction, true_mask)
    r = recall(prediction, true_mask)
    if precision == 0:
        return 0
    f1 = 2 * (p * r) / (p + r)
    return f1

import skimage
from skimage.morphology import thin
from collections import Counter

def mse(prediction, true_mask):
    mse_score = np.mean((prediction - true_mask) ** 2)
    return mse_score

def num_filopodia_blobs(mask):
    return skimage.measure.label(mask)

def num_filopodia_demerged(mask):
    thinned = thin(mask)
    img_thin_labeled = skimage.measure.label(thinned.astype(np.uint8), connectivity=2)
    stats_bbox = skimage.measure.regionprops(img_thin_labeled.astype(np.uint8))
    filopodia_count = 0
    for i in range(0, len(stats_bbox)):
        bbox = stats_bbox[i].bbox
        bbox_region = img_thin_labeled[bbox[0]:bbox[2], bbox[1]:bbox[3]]

        value_counts = Counter(bbox_region.flatten()).most_common()
        most_frequent_value = value_counts[1][0] if len(value_counts) > 1 else value_counts[0][0]
        bbox_region = (bbox_region == most_frequent_value) * 1

        # if into that bounding box #branchpoints > 1 AND #endpoints >= 4, it is a FUSED filopodia
        bbox_region_padded = np.pad(bbox_region, pad_width=4, mode='constant', constant_values=0)
        n_endpoints = find_endpoints(bbox_region_padded)
        
        filopodia_count += n_endpoints - 1
    return filopodia_count

def filopodia_length_sum(mask):
    return np.count_nonzero(thin(mask))

# calculate metrics

In [ ]:
IOUs, DICEs, PRECISIONs, RECALLs, F1SCOREs, MSEs = [],[],[],[],[],[]
filo_N_diffs, filo_N_abs_diffs, filo_len_diffs, filo_len_abs_diffs = [],[],[],[]
single_filo_N_diff, single_filo_N_abs_diff, merged_filo_N_diff, merged_filo_N_abs_diff = [],[],[],[]
test_set_indices = list(pd.read_excel("dataset/test_set_indices.xlsx")["id"])
img_directory = r"dataset\images"
mask_directory = r"dataset\masks"
pred_directory = r"toolsData\FiloAnalyzer320"
plt.rcParams["figure.figsize"] = (15,15)

IMG_HEIGHT, IMG_WIDTH = 320, 320


for i in tqdm.tqdm(range(1,244+1)):
    if not (i in test_set_indices):
        continue
    filename = f"{i}.tif"
    img = cv2.imread(os.path.join(img_directory, filename))
    mask = cv2.imread(os.path.join(mask_directory, filename))
    pred = cv2.imread(os.path.join(pred_directory, f"{i}.tif"))#.mean(axis=-1)

    # for Filopodyan
    # pred = (pred[:,:,0] >= pred.mean()) * 255


    # for FiloQuant
    # filename = os.listdir(r"C:\Users\ricca\Desktop\Thesis\toolsData\FiloQuant320" + "\\" + str(i) + "\\Tagged_skeleton_RGB")[0]
    # path = os.path.join(r"C:\Users\ricca\Desktop\Thesis\toolsData\FiloQuant320" + "\\" + str(i) + "\\Tagged_skeleton_RGB", filename)
    # pred = np.array(Image.open(path))
    # pred = ((pred[:,:,0] > 100) & (pred[:,:,1] < 100)).astype(np.uint8) * 255

    # for FiloDetect
    #pred = (pred[:,:,1] > pred.mean()) & (pred[:,:,2] < pred.mean())

    #print("preprocessing", end=" ")

    # preprocess the image
    if len(img.shape) > 2:
        img = img.mean(axis=-1)
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    img = img.reshape((1, IMG_HEIGHT, IMG_WIDTH, 1))

    # preprocess the mask
    if len(mask.shape) > 2:
        mask = mask.mean(axis=-1)
    mask = resize(mask, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    mask = (mask > 0.80) * 255

    # preprocess the pred
    if len(pred.shape) > 2:
        pred = pred.mean(axis=-1)
    pred = resize(pred, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    pred = (pred > 0.80) * 255

    # for the deep learning model
    pred = model.predict(img, verbose=0).reshape((IMG_HEIGHT, IMG_WIDTH))
    pred = cv2.erode(pred, np.ones((2, 2), np.uint8))
    pred = (pred > 0.5).astype(np.uint8) * 255

    # filter smallest contours
    # contours, _ = cv2.findContours(pred.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # filtered_contours = []
    # for contour in contours:
    #     area = cv2.contourArea(contour)
    #     if area > 25:
    #         filtered_contours.append(contour)
    # filtered_image = np.zeros_like(pred)
    # cv2.drawContours(filtered_image, filtered_contours, -1, 255, thickness=cv2.FILLED)
    # pred = filtered_image
    
    #print("detecting fused", end=" ")
    fused_pred, n_single_p, n_fused_p = detect_fused(pred)
    fused_true, n_single_t, n_fused_t = detect_fused(mask)
    #print("finish")


    # plt.imshow(pred), plt.show()
    # plt.imshow(mask), plt.show()

    IOUs.append(iou(pred, mask))
    DICEs.append(dice(pred, mask))
    PRECISIONs.append(precision(pred, mask))
    RECALLs.append(recall(pred, mask))
    F1SCOREs.append(f1_score(pred, mask))
    MSEs.append(mse(pred, mask))
    filo_N_diffs.append(num_filopodia_demerged(pred) - num_filopodia_demerged(mask))
    filo_N_abs_diffs.append(abs(num_filopodia_demerged(pred) - num_filopodia_demerged(mask)))
    filo_len_diffs.append(filopodia_length_sum(pred) - filopodia_length_sum(mask))
    filo_len_abs_diffs.append(abs(filopodia_length_sum(pred) - filopodia_length_sum(mask)))
    single_filo_N_diff.append(n_single_p - n_single_t)
    single_filo_N_abs_diff.append(abs(n_single_p - n_single_t))
    merged_filo_N_diff.append(n_fused_p - n_fused_t)
    merged_filo_N_abs_diff.append(abs(n_fused_p - n_fused_t))

# print results

In [ ]:
print("IOU", np.mean(IOUs), "±", np.std(IOUs))
print("DICE", np.mean(DICEs), "±", np.std(DICEs))
print("PRECISION", np.nanmean(PRECISIONs), "±", np.nanstd(PRECISIONs))
print("RECALL", np.mean(RECALLs), "±", np.std(RECALLs))
print("F1", np.nanmean(F1SCOREs), "±", np.nanstd(F1SCOREs))
print("MSE", np.mean(MSEs), "±", np.std(MSEs))
print("Filo # difference", np.mean(filo_N_diffs), "±", np.std(filo_N_diffs))
print("Filo # abs difference", np.mean(filo_N_abs_diffs), "±", np.std(filo_N_abs_diffs))
print("Filo len difference", np.mean(filo_len_diffs), "±", np.std(filo_len_diffs))
print("Filo len abs difference", np.mean(filo_len_abs_diffs), "±", np.std(filo_len_abs_diffs))
print("Single filo # diff", np.mean(single_filo_N_diff), "±", np.std(single_filo_N_diff))
print("Single filo # abs diff", np.mean(single_filo_N_abs_diff), "±", np.std(single_filo_N_abs_diff))
print("Fused filo # diff", np.mean(merged_filo_N_diff), "±", np.std(merged_filo_N_diff))
print("Fused filo # abs diff", np.mean(merged_filo_N_abs_diff), "±", np.std(merged_filo_N_abs_diff))
print(np.mean(IOUs), "±", np.std(IOUs), ",",
        np.mean(DICEs), "±", np.std(DICEs), ",",
        np.nanmean(PRECISIONs), "±", np.nanstd(PRECISIONs), ",",
        np.mean(RECALLs), "±", np.std(RECALLs), ",",
        np.nanmean(F1SCOREs), "±", np.nanstd(F1SCOREs), ",",
        np.mean(MSEs), "±", np.std(MSEs), ",",
        np.mean(filo_N_diffs), "±", np.std(filo_N_diffs), ",",
        np.mean(filo_N_abs_diffs), "±", np.std(filo_N_abs_diffs), ",",
        np.mean(filo_len_diffs), "±", np.std(filo_len_diffs), ",",
        np.mean(filo_len_abs_diffs), "±", np.std(filo_len_abs_diffs), ",",
        np.mean(single_filo_N_diff), "±", np.std(single_filo_N_diff) , ",",
        np.mean(single_filo_N_abs_diff), "±", np.std(single_filo_N_abs_diff) , ",",
        np.mean(merged_filo_N_diff), "±", np.std(merged_filo_N_diff) , ",",
        np.mean(merged_filo_N_abs_diff), "±", np.std(merged_filo_N_abs_diff) , ",",)